# Train Model

#### References
* https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
* https://pytorch.org/tutorials/beginner/saving_loading_models.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import models
import utils_train
import change_dataset_np
import matplotlib.pyplot as plt
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from IPython.display import clear_output, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Hyperparameters
num_epochs = 25
num_classes = 2
batch_size = 100
img_size = 224
base_lr = 1e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
batch_size *= num_gpu
base_lr *= num_gpu
print('Number of GPUs Available:', num_gpu)

train_pickle_file = 'change_dataset_train.pkl'
val_pickle_file = 'change_dataset_train.pkl'

PyTorch Version:  1.1.0
Torchvision Version:  0.3.0
Device: cuda:0
Number of GPUs Available: 8


#### Define Transformation

In [2]:
#transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()        
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor()        
    ]),
}

#### Load Dataset

In [3]:
# Create training and validation datasets
train_dataset = change_dataset_np.ChangeDatasetNumpy(train_pickle_file, data_transforms['train'])
val_dataset = change_dataset_np.ChangeDatasetNumpy(val_pickle_file, data_transforms['val'])
image_datasets = {'train': train_dataset, 'val': val_dataset}
# Create training and validation dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16)
#dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=8) for x in ['train', 'val']}
dataloaders_dict = {'train': train_loader, 'val': val_loader}

#### Start Tensorboard Interface

In [4]:
# Default directory "runs"
writer = SummaryWriter()

#### Initialize Model

In [5]:
img_reference_dummy = torch.randn(1,3,img_size,img_size)
img_test_dummy = torch.randn(1,3,img_size,img_size)
change_net = models.ChangeNet(num_classes=num_classes)

# Add on Tensorboard the Model Graph
writer.add_graph(change_net, [img_reference_dummy, img_test_dummy])

/mnt/anaconda3/lib/python3.7/site-packages/torch/onnx/symbolic.py:173: UserWarning: ONNX export failed on upsample_bilinear2d because align_corners == True not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


#### Send Model to GPUs (If Available)

In [6]:
if num_gpu > 1:
    change_net = nn.DataParallel(change_net)
change_net = change_net.to(device)

#### Load Weights

#### Initialize Loss Functions and Optimizers

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(change_net.parameters(), lr=base_lr)    
sc_plt = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, verbose=True)    

#### Train Model

In [ ]:
best_model, _ = utils_train.train_model(change_net, dataloaders_dict, criterion, optimizer, writer, device, num_epochs=num_epochs)
torch.save(best_model.state_dict(), './best_model.pkl')

Epoch 0/24
----------
train Loss: 1.0199
val Loss: 0.9525
Epoch 1/24
----------
train Loss: 0.9144
val Loss: 0.8898
Epoch 2/24
----------
train Loss: 0.8429
val Loss: 0.8255
Epoch 3/24
----------
train Loss: 0.7834
val Loss: 0.7773
Epoch 4/24
----------
train Loss: 0.7300
val Loss: 0.7183
Epoch 5/24
----------
train Loss: 0.6808
val Loss: 0.6717
Epoch 6/24
----------
train Loss: 0.6349
val Loss: 0.6251
Epoch 7/24
----------
train Loss: 0.5920
val Loss: 0.5807
Epoch 8/24
----------
train Loss: 0.5515
val Loss: 0.5376
Epoch 9/24
----------
train Loss: 0.5137
val Loss: 0.5066
Epoch 10/24
----------
train Loss: 0.4787
val Loss: 0.4639
Epoch 11/24
----------
train Loss: 0.4460
val Loss: 0.4351
Epoch 12/24
----------
train Loss: 0.4162
val Loss: 0.4063
Epoch 13/24
----------
train Loss: 0.3887
val Loss: 0.3742
Epoch 14/24
----------
train Loss: 0.3633
val Loss: 0.3536
Epoch 15/24
----------
train Loss: 0.3404
val Loss: 0.3269
Epoch 16/24
----------
train Loss: 0.3196
val Loss: 0.3058
Epoch 1

In [ ]:
@interact(idx=widgets.IntSlider(min=0,max=len(val_dataset)-1))
def explore_validation_dataset(idx):
    best_model.eval()
    sample = val_dataset[idx]
    reference = sample['reference'].unsqueeze(0)
    test = sample['test'].unsqueeze(0)
    #label = sample['label'].type(torch.LongTensor).squeeze(0).cpu().numpy()
    label = (sample['label']>0).type(torch.LongTensor).squeeze(0).cpu().numpy()
    pred = best_model([reference, test])
    #print(pred.shape)
    _, output = torch.max(pred, 1)
    output = output.squeeze(0).cpu().numpy()
    fig=plt.figure(figsize=(16, 16))
    #fig.add_subplot(1, 2, 1)
    #plt.imshow(reference)
    #fig.add_subplot(1, 4, 2)
    #plt.imshow(test)
    fig.add_subplot(1, 2, 1)
    plt.imshow(label)
    fig.add_subplot(1, 2, 2)
    plt.imshow(output)
    plt.show()
    print(np.max(label))
    print(np.max(output))